In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

In [3]:
# k
ks = [5, 10, 50]

score_function_dict = {
    "Recall"      : evaluators.recall,
    "Unpopularity": evaluators.unpopularity,
    "Unpopularity2": evaluators.unpopularity2,
    "F1-score"    : evaluators.f1_score
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
coverage = evaluators.CoverageEvaluator(test_set, ks)
hubness = evaluators.HubnessEvaluator(test_set, ks)

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 30
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = False

search_range = 30

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma)
mymp = searches.Mymp(model, search_range)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [6]:
trainer.fit(n_batch, n_epoch, knn, userwise, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 940/940 [00:09<00:00, 96.45it/s]


In [7]:
trainer.valid_scores

,Recall@5,Unpopularity@5,Unpopularity2@5,F1-score@5,Recall@10,Unpopularity@10,Unpopularity2@10,F1-score@10,Recall@50,Unpopularity@50,Unpopularity2@50,F1-score@50,epoch,losses
0,0.067529,0.969628,6.661310,0.103739,0.126793,0.969657,6.660967,0.188023,0.526501,0.917427,6.290596,0.578800,0,NaN
0,0.211210,0.905919,3.819723,0.292408,0.383806,0.899299,3.739618,0.481708,0.814228,0.883376,4.657983,0.821936,10,0.456912
0,0.324261,0.877169,3.339761,0.420305,0.489081,0.886838,3.542301,0.584311,0.850714,0.884402,4.618094,0.846692,20,0.306798
0,0.374671,0.878111,3.347455,0.469409,0.534859,0.890269,3.635315,0.622091,0.873822,0.886836,4.707256,0.862463,30,0.256478


In [ ]:
trainer.valid(mymp, userwise)

In [ ]:
trainer.valid_scores

In [ ]:
mp2 = searches.MutualProximity(model, gaussian)

In [ ]:
trainer.valid(mp, recall)

In [ ]:
trainer.valid_scores